# Route optimization for Grupo Coppel

## Obtaining the data

In [1]:
import os
from pathlib import Path

import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import polyline
import requests
from folium import Marker
from matplotlib import cm
from ortools.constraint_solver import pywrapcp, routing_enums_pb2


In [2]:
path = Path('../data/raw/UbicacionesVolumen.csv') # use S3 instead

In [3]:
df_ubicaciones = pd.read_csv(path)
#df_ubicaciones

## Adding the CEDIS and changing the volume 

In [4]:
# CEDIS Azcapotzalco location
df_ubicaciones=df_ubicaciones.append({"index":00000,"Latitud":19.4907364,"Longitud":-99.1620038,"Direccion":"CEDIS Azcapotzcalco","Vol":0},ignore_index=True)

# Changing the m^3 to cm^3
df_ubicaciones["Vol"]=round(df_ubicaciones["Vol"]*1000000)

# Creating the Delivery Time col
df_ubicaciones["TiempoEntrega"]=0

# This is an estimate for the time it would take (in seconds), depending of the load
df_ubicaciones.loc[df_ubicaciones['Vol']<=500000,"TiempoEntrega"]=5*60
df_ubicaciones.loc[(df_ubicaciones['Vol']>500000)&(df_ubicaciones['Vol']<=1000000),"TiempoEntrega"]=10*60
df_ubicaciones.loc[(df_ubicaciones['Vol']>1000000)&(df_ubicaciones['Vol']<=2000000),"TiempoEntrega"]=20*60
df_ubicaciones.loc[(df_ubicaciones['Vol']>2000000)&(df_ubicaciones['Vol']<=3000000),"TiempoEntrega"]=25*60
df_ubicaciones.loc[(df_ubicaciones['Vol']>3000000),"TiempoEntrega"]=30*60

# We obtain the next df
#df_ubicaciones

## Estimating the time

We concat every coordinate on a string

In [5]:
coordenadas=""
for i in range(len(df_ubicaciones)):
    fila=df_ubicaciones.iloc[i]
    long=fila["Longitud"]
    lat=fila["Latitud"]
    coordenadas=coordenadas+str(long)+","+str(lat)+";"
coordenadas=coordenadas[:-1]
#coordenadas

Creating an empty dataframe for the time

In [6]:
df_tiempos = pd.DataFrame()

We are going to use the next API http://project-osrm.org/

In [7]:
for i in range(0,291,10):
    rangos= list(range(i,i+10))
    sources= ";".join(str(item) for item in rangos)
    url="http://router.project-osrm.org/table/v1/car/"+coordenadas+"?sources="+sources
    r = requests.get(url)
    res = r.json()
    df_tiempos=df_tiempos.append(res["durations"])
    print("Ubicaciones Completas:",i+10)
url="http://router.project-osrm.org/table/v1/car/"+coordenadas+"?sources=300"
r = requests.get(url)
res = r.json()
df_tiempos=df_tiempos.append(res["durations"])
#df_tiempos

Ubicaciones Completas: 10
Ubicaciones Completas: 20
Ubicaciones Completas: 30
Ubicaciones Completas: 40
Ubicaciones Completas: 50
Ubicaciones Completas: 60
Ubicaciones Completas: 70
Ubicaciones Completas: 80
Ubicaciones Completas: 90
Ubicaciones Completas: 100
Ubicaciones Completas: 110
Ubicaciones Completas: 120
Ubicaciones Completas: 130
Ubicaciones Completas: 140
Ubicaciones Completas: 150
Ubicaciones Completas: 160
Ubicaciones Completas: 170
Ubicaciones Completas: 180
Ubicaciones Completas: 190
Ubicaciones Completas: 200
Ubicaciones Completas: 210
Ubicaciones Completas: 220
Ubicaciones Completas: 230
Ubicaciones Completas: 240
Ubicaciones Completas: 250
Ubicaciones Completas: 260
Ubicaciones Completas: 270
Ubicaciones Completas: 280
Ubicaciones Completas: 290
Ubicaciones Completas: 300


Change the index so its the same as in the original df

In [8]:
df_tiempos.columns=df_ubicaciones["index"]
df_tiempos.index=df_ubicaciones["index"]
#df_tiempos

Calculate the time

In [9]:
volumen5=df_ubicaciones[df_ubicaciones["Vol"]<=500000]["index"]
volumen10=df_ubicaciones[(df_ubicaciones["Vol"]>500000) &(df_ubicaciones["Vol"]<=1000000)]["index"]
volumen20=df_ubicaciones[(df_ubicaciones["Vol"]>1000000) &(df_ubicaciones["Vol"]<=2000000)]["index"]
volumen25=df_ubicaciones[(df_ubicaciones["Vol"]>2000000) &(df_ubicaciones["Vol"]<=3000000)]["index"]
volumen30=df_ubicaciones[df_ubicaciones["Vol"]>3000000]["index"]
volumen30

137    24156
138    24164
139    24439
250    24559
Name: index, dtype: int64

In [10]:
df_tiempos[volumen5]=df_tiempos[volumen5]+(5*60)
df_tiempos[volumen10]=df_tiempos[volumen10]+(10*60)
df_tiempos[volumen20]=df_tiempos[volumen20]+(20*60)
df_tiempos[volumen25]=df_tiempos[volumen25]+(25*60)
df_tiempos[volumen30]=df_tiempos[volumen30]+(30*60)
df_tiempos

index,23813,23815,23817,23956,23821,23822,23827,23830,23837,23838,...,24735,24740,24742,24750,24753,24756,24757,24762,24764,0
index,,,,,,,,,,,,,,,,,,,,,
23813,600.0,1340.2,2254.3,1772.8,1908.7,1347.4,1843.8,1163.2,653.9,1620.9,...,1551.0,1107.9,1313.8,1599.5,1151.3,1466.9,1328.6,1110.3,1208.0,1170.5
23815,1249.9,600.0,1668.1,1498.9,1559.6,770.6,1853.2,1492.6,997.0,1034.7,...,1770.8,1305.2,1540.4,1823.4,1333.7,1572.9,915.8,524.1,667.5,1228.3
23817,1433.0,900.8,1200.0,1682.0,1616.2,1025.4,2036.3,1675.7,1180.1,942.3,...,1953.9,1488.3,1723.5,2006.5,1516.8,1756.0,1098.9,653.8,747.1,1411.4
23956,1210.2,995.5,1909.6,1200.0,1612.2,1007.4,1582.8,1235.4,863.8,1276.2,...,1500.4,1034.8,1270.0,1553.0,1063.3,1423.1,1024.4,765.6,868.0,978.6
23821,2210.3,1897.0,2392.9,2471.0,300.0,1996.2,2539.1,2225.2,1895.6,2093.4,...,2261.3,1770.3,2036.4,2248.9,1813.7,1945.4,1454.2,1708.5,1868.2,1617.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24756,1724.7,1912.0,2826.1,2338.6,2001.0,1985.9,2064.3,1311.0,1170.2,2192.7,...,701.1,789.1,548.4,794.6,763.8,300.0,1860.1,1682.1,1846.5,1242.9
24757,1504.3,1195.2,2146.5,1769.6,1570.4,1294.4,2057.9,1759.5,1263.9,1513.1,...,1996.1,1515.7,1771.2,1983.7,1559.1,1680.2,300.0,1002.5,1166.4,1392.0
24762,1386.1,824.4,1644.0,1635.1,1695.8,949.0,1989.4,1628.8,1133.2,1010.6,...,1907.0,1441.4,1676.6,1959.6,1469.9,1709.1,1052.0,300.0,759.4,1364.5


Convertirlo a una lista

In [11]:
tiemposMatrix= df_tiempos.values.tolist()

## CVRP Model

In [12]:
available_trucks = 20

In [13]:
capacidades = np.repeat(20000000,available_trucks)

In [14]:
orden=[]
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = tiemposMatrix
    data['demands'] = df_ubicaciones['Vol']
    data['vehicle_capacities'] = capacidades
    data['num_vehicles'] = len(capacidades)
    data['depot'] = len(df_ubicaciones)-1
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        entregas=[]
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            entregas.append(node_index)
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Time of the route: {} seconds\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
        orden.append(entregas)
    print('Total time of all routes: {} seconds'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    orden=[]


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(30)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    
    return(routing)


routing= main()

Objective: 233365
Route for vehicle 0:
 300 Load(0.0) ->  300 Load(0.0)
Time of the route: 0 seconds
Load of the route: 0.0

Route for vehicle 1:
 300 Load(0.0) ->  300 Load(0.0)
Time of the route: 0 seconds
Load of the route: 0.0

Route for vehicle 2:
 300 Load(0.0) ->  300 Load(0.0)
Time of the route: 0 seconds
Load of the route: 0.0

Route for vehicle 3:
 300 Load(0.0) ->  300 Load(0.0)
Time of the route: 0 seconds
Load of the route: 0.0

Route for vehicle 4:
 300 Load(0.0) ->  300 Load(0.0)
Time of the route: 0 seconds
Load of the route: 0.0

Route for vehicle 5:
 300 Load(0.0) ->  300 Load(0.0)
Time of the route: 0 seconds
Load of the route: 0.0

Route for vehicle 6:
 300 Load(0.0) ->  300 Load(0.0)
Time of the route: 0 seconds
Load of the route: 0.0

Route for vehicle 7:
 300 Load(0.0) ->  300 Load(0.0)
Time of the route: 0 seconds
Load of the route: 0.0

Route for vehicle 8:
 300 Load(0.0) ->  300 Load(0.0)
Time of the route: 0 seconds
Load of the route: 0.0

Route for vehicle 9

In [15]:
ordenp=[]
capacidadF=[]
for i in range(len(orden)):
    temp1=orden[i]
    if(len(temp1)!=1):
        temp= temp1
        temp.append(temp[0])
        ordenp.append(temp)
        capacidadF.append(capacidades[i])
for j in range(len(ordenp)):
    i=ordenp[j]
    print("Camión",j,"hace",len(i),"paradas")

Camión 0 hace 31 paradas
Camión 1 hace 34 paradas
Camión 2 hace 43 paradas
Camión 3 hace 32 paradas
Camión 4 hace 38 paradas
Camión 5 hace 56 paradas
Camión 6 hace 46 paradas
Camión 7 hace 36 paradas


In [16]:
listdf_enrutado=[]
for i in ordenp:
    df_enrutado=pd.DataFrame(columns=df_ubicaciones.columns)
    df_enrutado=df_enrutado.append(df_ubicaciones.iloc[i])
    df_enrutado.reset_index(inplace=True)
    listdf_enrutado.append(df_enrutado)
print("Cantidad de dataframes:",len(listdf_enrutado))

Cantidad de dataframes: 8


In [17]:
for i in range(len(listdf_enrutado)):
    df_temp=listdf_enrutado[i]
    sum_volumen=df_temp["Vol"].sum()
    print("Camión ",i,"volumen=",sum_volumen,"cm^3 de",capacidadF[i],"cm^3")

Camión  0 volumen= 19880315.0 cm^3 de 20000000 cm^3
Camión  1 volumen= 19781221.0 cm^3 de 20000000 cm^3
Camión  2 volumen= 18563690.0 cm^3 de 20000000 cm^3
Camión  3 volumen= 19986003.0 cm^3 de 20000000 cm^3
Camión  4 volumen= 19028919.0 cm^3 de 20000000 cm^3
Camión  5 volumen= 19970970.0 cm^3 de 20000000 cm^3
Camión  6 volumen= 19936438.0 cm^3 de 20000000 cm^3
Camión  7 volumen= 19898067.0 cm^3 de 20000000 cm^3


In [18]:
listdf_rutasFinales=[]
indexes=df_ubicaciones["index"]
for j in ordenp:
  df_rutasFinales=pd.DataFrame(columns=["Inicio","Final"])
  for i in range(len(j)-1):
    inicio=i
    indexInicio=indexes[j[inicio]]
    final=i+1
    indexFinal=indexes[j[final]]
    fila={"Inicio":indexInicio,"Final":indexFinal}
    df_rutasFinales=df_rutasFinales.append(fila,ignore_index=True)
  listdf_rutasFinales.append(df_rutasFinales)
print("Cantidad de Dataframes:",len(listdf_rutasFinales))

Cantidad de Dataframes: 8


In [19]:
list_columnaRutas=[]
list_columnaTiempos=[]
contador=0
for i in ordenp:
  columnaRutas=[]
  columnaTiempos=[]
  for j in range(len(i)-1):
    ubicacion1=df_ubicaciones.iloc[i[j]]
    ubicacion2=df_ubicaciones.iloc[i[j+1]]
    coordenadas=str(ubicacion1["Longitud"])+","+str(ubicacion1["Latitud"])+";"+str(ubicacion2["Longitud"])+","+str(ubicacion2["Latitud"])
    url="http://router.project-osrm.org/route/v1/foot/"+coordenadas
    r = requests.get(url)
    res = r.json()
    columnaRutas.append(polyline.decode(res["routes"][0]["geometry"]))
    columnaTiempos.append(res["routes"][0]["duration"])
  list_columnaRutas.append(columnaRutas)
  list_columnaTiempos.append(columnaTiempos)
  contador+=1
  print("Camión",contador,"Terminado")

Camión 1 Terminado
Camión 2 Terminado
Camión 3 Terminado
Camión 4 Terminado
Camión 5 Terminado
Camión 6 Terminado
Camión 7 Terminado
Camión 8 Terminado


In [20]:
for i in range(len(listdf_rutasFinales)):
  df_temp= listdf_rutasFinales[i]
  df_temp["Ruta"]=list_columnaRutas[i]
  df_temp["Tiempo"]=list_columnaTiempos[i]
  listdf_rutasFinales[i]=df_temp
print("Cantidad de Dataframes:",len(listdf_rutasFinales))

Cantidad de Dataframes: 8


In [21]:
for i in range(len(listdf_rutasFinales)):
  df_temp=listdf_rutasFinales[i]
  sum_tiempo=df_temp["Tiempo"].sum()
  print("Camión ",i,"tiempo=",round(sum_tiempo),"segundos |", round(sum_tiempo/60),"minutos")

Camión  0 tiempo= 6513 segundos | 109 minutos
Camión  1 tiempo= 7638 segundos | 127 minutos
Camión  2 tiempo= 7841 segundos | 131 minutos
Camión  3 tiempo= 5995 segundos | 100 minutos
Camión  4 tiempo= 6385 segundos | 106 minutos
Camión  5 tiempo= 13185 segundos | 220 minutos
Camión  6 tiempo= 13239 segundos | 221 minutos
Camión  7 tiempo= 7416 segundos | 124 minutos


In [22]:
def gradientColors(nColors,gradientN):
  gradiente=cm.get_cmap(gradientN,256)
  listaColores=[]
  for i in np.linspace(0,1,nColors):
    listaColores.append(gradiente(i))
  return(listaColores)

In [23]:
listaColores= gradientColors(len(listdf_enrutado),"gist_rainbow")
listaColoresHex=[]
for i in range(len(listaColores)):
  colorTuple= listaColores[i]
  colorHex= '#%02x%02x%02x' % tuple((np.array(colorTuple)*255).astype(int)[:-1])
  listaColoresHex.append(colorHex)
listaColoresHex

['#ff0028',
 '#ff9900',
 '#9cff00',
 '#00ff25',
 '#00ffec',
 '#004dff',
 '#7b00ff',
 '#ff00bf']

In [24]:
for i in range(len(listdf_enrutado)):
  df_tempEnrutado=listdf_enrutado[i]
  df_tempEnrutado["Color"]=listaColoresHex[i]
  listdf_enrutado[i]=df_tempEnrutado
  df_tempRutas=listdf_rutasFinales[i]
  df_tempRutas["Color"]=listaColoresHex[i]
  listdf_rutasFinales[i]=df_tempRutas
  print("Camion",i,"es de color",listaColoresHex[i])

Camion 0 es de color #ff0028
Camion 1 es de color #ff9900
Camion 2 es de color #9cff00
Camion 3 es de color #00ff25
Camion 4 es de color #00ffec
Camion 5 es de color #004dff
Camion 6 es de color #7b00ff
Camion 7 es de color #ff00bf


In [25]:
map= folium.Map(location=[19.4907364,-99.1620038],zoom_start=12, tiles="cartodbpositron")
for j in listdf_enrutado:
  for index, row in j.iloc[:-1].iterrows():
    folium.CircleMarker(location=[row["Latitud"],row["Longitud"]],popup=str(index)+": "+row["Direccion"],
                        fill=True,fill_color=row["Color"],color = row["Color"],fill_opacity=0.7, 
                        tooltip=("Tiempo Entrega: "+str(row["TiempoEntrega"]/60)+" minutos")).add_to(map)


Marker([19.4907364,-99.1620038],icon=folium.Icon(color="darkblue",icon_color="white",icon="glyphicon-home"),tooltip="CEDIS Azcapotzalco").add_to(map)
map

In [26]:
for i in listdf_rutasFinales:
  for index, row in i.iterrows():
    folium.PolyLine(row["Ruta"],weight=8,color=row["Color"],opacity=0.6,tooltip=str(index)+": Tiempo enrutamiento "+str(round(row["Tiempo"]/60))+" minutos").add_to(map)
map

In [27]:
map.save("CRVP_Final_TiemposEntrega.html")